# Tool Use

The `AgentChat` api provides a `ToolUseAssistantAgent` with presets for adding tools that the agent can call as part of it's response. 

:::{note}

The example presented here is a work in progress 🚧. Also, tool uses here assumed the `model_client` used by the agent supports tool calling. 
::: 

In [1]:
from autogen_agentchat.agents import ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

In AgentChat, a Tool is a function wrapped in the `FunctionTool` class exported from `autogen_core.components.tools`.   

In [2]:
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."


get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

Finally, agents that use tools are defined in the following manner. 

-  An agent is instantiated based on the  `ToolUseAssistantAgent` class in AgentChat. The agent is aware of the tools it can use by passing a `tools_schema` attribute to the class, which is passed to the `model_client` when the agent generates a response.
-  An agent Team is defined that takes a list of `tools`.  Effectively, the `ToolUseAssistantAgent` can generate messages that call tools, and the team is responsible executing those tool calls and returning the results.

In [3]:
assistant = ToolUseAssistantAgent(
    "Weather_Assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    registered_tools=[get_weather_tool],
)
team = RoundRobinGroupChat([assistant])
result = await team.run("What's the weather in New York?")
# print(result)


--------------------------------------------------------------------------- 
[2024-10-08T09:50:13.202461]:

What's the weather in New York?
From: user
--------------------------------------------------------------------------- 
[2024-10-08T09:50:14.090696], Weather_Assistant:

[FunctionCall(id='call_wqkaIBdYjWklWG0GQkYz7FZ0', arguments='{"city":"New York"}', name='get_weather')]
From: Weather_Assistant
--------------------------------------------------------------------------- 
[2024-10-08T09:50:14.092050], tool_agent_for_Weather_Assistant:

[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', call_id='call_wqkaIBdYjWklWG0GQkYz7FZ0')]
From: tool_agent_for_Weather_Assistant
--------------------------------------------------------------------------- 
[2024-10-08T09:50:14.714470], Weather_Assistant:

The weather in New York is 72 degrees and sunny. 

TERMINATE
From: Weather_Assistant

## Using Langchain Tools 

AutoGen also provides direct support for tools from LangChain via the `autogen_ext`  package.



In [4]:
# pip install langchain, langchain-community, wikipedia , autogen-ext

from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

langchain_wikipedia_tool = LangChainToolAdapter(tool)

In [8]:
wikipedia_assistant = ToolUseAssistantAgent(
    "WikiPedia_Assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    registered_tools=[langchain_wikipedia_tool],
)
team = RoundRobinGroupChat([wikipedia_assistant])
result = await team.run("Who was the first president of the United States?")

# print(result)


--------------------------------------------------------------------------- 
[2024-10-08T09:51:36.869317]:

Who is the receipient of the 2023 Nobel Prize in Physics?
From: user


--------------------------------------------------------------------------- 
[2024-10-08T09:51:37.856066], WikiPedia_Assistant:

[FunctionCall(id='call_bdLqS1msbHCy5IMGYaata5vs', arguments='{"query":"2023 Nobel Prize in Physics"}', name='wikipedia')]
From: WikiPedia_Assistant
--------------------------------------------------------------------------- 
[2024-10-08T09:51:38.518288], tool_agent_for_WikiPedia_Assistant:

[FunctionExecutionResult(content='Page: Nobel Prize in Physics\nSummary: The Nobel Prize in Physics (Swedish: Nobelpriset i fysik) is a', call_id='call_bdLqS1msbHCy5IMGYaata5vs')]
From: tool_agent_for_WikiPedia_Assistant
--------------------------------------------------------------------------- 
[2024-10-08T09:51:39.070911], WikiPedia_Assistant:

[FunctionCall(id='call_BFXGGeuBbOQ1LPb4f0NiNva2', arguments='{"query":"2023 Nobel Prize in Physics recipients"}', name='wikipedia')]
From: WikiPedia_Assistant
---------------------------------------------------------------------